In [14]:
import glob
import pandas as pd
import os
import matplotlib.pyplot as plt
import math

In [15]:
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.optim as optim
import torch.nn as nn

In [16]:
data_clean_folder = "stage3_data_cleaning/v2"
folder_path = data_clean_folder
xlsx_files = glob.glob(os.path.join(folder_path, '*.xlsx'))
xlsx_files

['stage3_data_cleaning/v2\\type10_label_merged_final_decoded_clean3.xlsx',
 'stage3_data_cleaning/v2\\type1_label_merged_final_decoded_clean3.xlsx',
 'stage3_data_cleaning/v2\\type2_label_merged_final_decoded_clean3.xlsx',
 'stage3_data_cleaning/v2\\type3_label_merged_final_decoded_clean3.xlsx',
 'stage3_data_cleaning/v2\\type4_label_merged_final_decoded_clean3.xlsx',
 'stage3_data_cleaning/v2\\type5_label_merged_final_decoded_clean3.xlsx',
 'stage3_data_cleaning/v2\\type6_label_merged_final_decoded_clean3.xlsx',
 'stage3_data_cleaning/v2\\type7_label_merged_final_decoded_clean3.xlsx',
 'stage3_data_cleaning/v2\\type8_label_merged_final_decoded_clean3.xlsx',
 'stage3_data_cleaning/v2\\type9_label_merged_final_decoded_clean3.xlsx']

In [17]:
# df=pd.read_excel(xlsx_files[1])
# df.columns

In [18]:
# train_threshold = math.ceil(0.6 * max(df['timestamp']))
# val_threshold = math.ceil(0.8 * max(df['timestamp']))
# train_threshold,val_threshold

In [19]:
# data=df

In [20]:
# train_data = data[data['timestamp'] <= train_threshold]
# val_data = data[(data['timestamp'] > train_threshold) & (data['timestamp'] <= val_threshold)]
# test_data = data[data['timestamp'] > val_threshold]

In [21]:
# X_train = train_data.drop(['label', 'timestamp','can_id'], axis=1)
# y_train = train_data['label']
# X_val = val_data.drop(['label', 'timestamp','can_id'], axis=1)
# y_val = val_data['label']
# X_test = test_data.drop(['label', 'timestamp','can_id'], axis=1)
# y_test = test_data['label']

In [22]:
# scaler = MinMaxScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_val_scaled = scaler.transform(X_val)
# X_test_scaled = scaler.transform(X_test)

In [23]:
# X_train_lstm = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
# X_val_lstm = X_val_scaled.reshape((X_val_scaled.shape[0], 1, X_val_scaled.shape[1]))
# X_test_lstm = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))


In [24]:
# import torch
# X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32).unsqueeze(1)
# y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1)
# X_val_tensor = torch.tensor(X_val_scaled, dtype=torch.float32).unsqueeze(1)
# y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32).unsqueeze(1)
# X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32).unsqueeze(1)
# y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).unsqueeze(1)

# print(f"Train set: {X_train_tensor.shape}")
# print(f"Validation set: {X_val_tensor.shape}")
# print(f"Test set: {X_test_tensor.shape}")

In [25]:


class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        out = self.sigmoid(out)
        return out

# input_size = X_train_tensor.shape[2]
# hidden_size = 50
# num_layers = 2
# output_size = 1

# model = LSTMModel(input_size, hidden_size, num_layers, output_size)
# print(model)


In [38]:
from sklearn.metrics import mean_squared_error,accuracy_score

def lstm_approach(X_train_tensor,y_train_tensor,X_val_tensor,y_val_tensor,X_test_tensor,y_test_tensor):

    input_size = X_train_tensor.shape[2]
    hidden_size = 50
    num_layers = 2
    output_size = 1

    model = LSTMModel(input_size, hidden_size, num_layers, output_size)
    print(model)
    
    # criterion = nn.MSELoss() # Regression LOSS
    criterion = nn.BCELoss()
    
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    num_epochs = 100
    train_accuracies = []
    val_accuracies = []
    
    for epoch in range(num_epochs):
        model.train()
        outputs = model(X_train_tensor)
        optimizer.zero_grad()
        loss = criterion(outputs, y_train_tensor)
        loss.backward()
        optimizer.step()
        
        model.eval()
        val_outputs = model(X_val_tensor)
        val_loss = criterion(val_outputs, y_val_tensor)
        val_predictions = (val_outputs > 0.5).float()
        val_accuracy = accuracy_score(y_val_tensor.cpu(), val_predictions.cpu())
        
        if (epoch+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}, Val Accuracy: {val_accuracy:.4f}')

    model.eval()
    test_outputs = model(X_test_tensor)
    test_loss = criterion(test_outputs, y_test_tensor)
    test_predictions = (test_outputs > 0.5).float()
    test_accuracy = accuracy_score(y_test_tensor.cpu(), test_predictions.cpu())


    val_outputs = model(X_val_tensor)
    val_loss = criterion(val_outputs, y_val_tensor)
    val_predictions = (val_outputs > 0.5).float()
    val_accuracy = accuracy_score(y_val_tensor.cpu(), val_predictions.cpu())

    return val_loss.item(),test_loss.item(),test_accuracy,val_accuracy,model


In [39]:
def feature_analysis_lstm(data):
    train_threshold = math.ceil(0.6 * max(data['timestamp']))
    val_threshold = math.ceil(0.8 * max(data['timestamp']))

    train_data = data[data['timestamp'] <= train_threshold]
    val_data = data[(data['timestamp'] > train_threshold) & (data['timestamp'] <= val_threshold)]
    test_data = data[data['timestamp'] > val_threshold]

    X_train = train_data.drop(['label', 'timestamp','can_id'], axis=1)
    y_train = train_data['label']
    X_val = val_data.drop(['label', 'timestamp','can_id'], axis=1)
    y_val = val_data['label']
    X_test = test_data.drop(['label', 'timestamp','can_id'], axis=1)
    y_test = test_data['label']

    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    X_test_scaled = scaler.transform(X_test)

    X_train_lstm = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
    X_val_lstm = X_val_scaled.reshape((X_val_scaled.shape[0], 1, X_val_scaled.shape[1]))
    X_test_lstm = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

    X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32).unsqueeze(1)
    y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1)
    X_val_tensor = torch.tensor(X_val_scaled, dtype=torch.float32).unsqueeze(1)
    y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32).unsqueeze(1)
    X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32).unsqueeze(1)
    y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).unsqueeze(1)
    
    print(f"Train set: {X_train_tensor.shape}")
    print(f"Validation set: {X_val_tensor.shape}")
    print(f"Test set: {X_test_tensor.shape}")

    return X_train_tensor,y_train_tensor,X_val_tensor,y_val_tensor,X_test_tensor,y_test_tensor
    



    

In [40]:
results_list = []
for file_path in xlsx_files:
    # print(file_path)
    results = {}
    label_key = os.path.basename(file_path).split('.')[0][:-28]
    print(label_key)
    results['dataset'] = label_key
    
    data = pd.read_excel(file_path)
    
    X_train,y_train,X_val,y_val, X_test,y_test = feature_analysis_lstm(data)

    val_loss, test_loss,test_accuracy,val_accuracy,model = lstm_approach(X_train,y_train,X_val,y_val, X_test,y_test)
    
    results['LSTM_VAL_LOSS'] = val_loss
    results['LSTM_VAL_ACC'] = val_accuracy
    results['LSTM_Test_LOSS'] = test_loss
    results['LSTM_Test_ACC'] = test_accuracy

    results_list.append(results)
    # results_df = results_df.append(results, ignore_index=True)
    print(results)

results_df = pd.DataFrame(results_list)


type10_label
Train set: torch.Size([110047, 1, 19])
Validation set: torch.Size([58848, 1, 19])
Test set: torch.Size([38485, 1, 19])
LSTMModel(
  (lstm): LSTM(19, 50, num_layers=2, batch_first=True)
  (fc): Linear(in_features=50, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)
Epoch [10/100], Loss: 0.6710, Val Loss: 0.7026, Val Accuracy: 0.4488
Epoch [20/100], Loss: 0.6546, Val Loss: 0.7141, Val Accuracy: 0.4488
Epoch [30/100], Loss: 0.6335, Val Loss: 0.7392, Val Accuracy: 0.4488
Epoch [40/100], Loss: 0.6108, Val Loss: 0.7962, Val Accuracy: 0.4488
Epoch [50/100], Loss: 0.6015, Val Loss: 0.8835, Val Accuracy: 0.4488
Epoch [60/100], Loss: 0.6003, Val Loss: 0.8921, Val Accuracy: 0.4488
Epoch [70/100], Loss: 0.5969, Val Loss: 0.8537, Val Accuracy: 0.4488
Epoch [80/100], Loss: 0.5943, Val Loss: 0.8435, Val Accuracy: 0.4488
Epoch [90/100], Loss: 0.5907, Val Loss: 0.8495, Val Accuracy: 0.4488
Epoch [100/100], Loss: 0.5862, Val Loss: 0.8427, Val Accuracy: 0.4488
{'dataset': 'type10_label', 

In [24]:
# val_loss.item()

0.1786368489265442

In [41]:
results_df

,dataset,LSTM_VAL_LOSS,LSTM_VAL_ACC,LSTM_Test_LOSS,LSTM_Test_ACC
0,type10_label,0.842691,0.448783,0.607108,0.697908
1,type1_label,0.225007,1.000000,0.422737,0.819507
2,type2_label,0.574738,0.700146,0.463296,0.799758
3,type3_label,0.363148,0.922017,0.334697,0.931749
4,type4_label,0.571547,0.730036,0.471944,0.812117
5,type5_label,0.585255,0.704583,0.588846,0.706348
6,type6_label,0.664721,0.588834,0.619928,0.643566
7,type7_label,0.611358,0.667814,0.586895,0.626021
8,type8_label,0.554610,0.710202,0.519994,0.695926
9,type9_label,0.571347,0.693425,0.615654,0.685863


In [31]:
output_file = 'results_LSTM_v2.xlsx'
results_df.to_excel(output_file, index=False)

In [ ]:
import torch.optim as optim

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 200
for epoch in range(num_epochs):
    model.train()
    outputs = model(X_train_tensor)
    optimizer.zero_grad()
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    
    model.eval()
    val_outputs = model(X_val_tensor)
    val_loss = criterion(val_outputs, y_val_tensor)
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}')


In [ ]:
model.eval()
test_outputs = model(X_test_tensor)
test_loss = criterion(test_outputs, y_test_tensor)

print(f'Test MSE: {test_loss.item():.4f}')
